## Eclipse Prediction

Use SkyField to find eclipses in a given time period, generate besellian elements and plot path of
selected eclipses

### General approach

As with all eclipse, brute force searching is required.

1. Find eclipse candidates by searching with coarse time interval
2. For each candidate, do a more precise evaluation
3. generate Besselians
4. For a give eclipse, plot the path using cartopy

### Finding Candidates for total/annular eclipse (first cut excludes partial/penumbral)

For the time interval selected, determinme whether the vector from sun through moon
intersects a plane through the centre of the earth. Becuase we want to do Besselian, we use that plane definition, which is:

> Definition: An imaginary plane passing through the center of the Earth, oriented perpendicular to the axis of the Moon's shadow (the line connecting the centers of the Sun and Moon) at a specific instant

However, looking at `eclipselib.py` a better approach would be to look at the shadow cones after finding mimima earth moon sun angle

The intersection of the SM vector with the plane is found by using the EM vector. fundamental plane normal vector from the centre of the earth is parallel to the actual shadow vector. We want just 1 component from that in the plane (the part perpendicular to the plane)

```
         ^ z_fp (Shadow Axis Direction; Normal to Fund. Plane)
          |
          |
          |                 M (Moon's Center)
          |                /.
          |               / .
          |              /  .
          |             /   .  <-- Shadow Axis Line (Parallel to z_fp)
          |            /    .
          |           /     .
          | m_vec    /      .
          |         /       .
          |        /        .
          |       /         .
  --------O------------------P_fp--------------------> y_fp (or x_fp) Axis
 (Earth's |      /          |                        (Fundamental Plane edge-on)
 Center)  |     /           | m_parallel
          |    /            | (Component || z_fp)
          |   /             |
          |  /              V
          | /               M_proj (Projection of M onto Fund. Plane)
          |/
          ------------------->
            m_perpendicular
            (Component in Fund. Plane)
            Vector O -> P_fp
```


In [69]:
# Setup Imports
import numpy as np
from skyfield.api import load, N, W, E, S, PlanetaryConstants # For units if needed
from skyfield import almanac
from skyfield.units import Angle
from scipy.optimize import minimize_scalar
from matplotlib import pyplot as plt

import datetime

# common variables an ephemeris etc
# Load ephemeris and timescale
ts = load.timescale()
# Using a more recent ephemeris like de440.bsp is generally recommended
try:
    eph = load('de440.bsp')
except Exception:
    print("Could not load de440.bsp, falling back to de421.bsp")
    eph = load('de421.bsp')

SUN_RADIUS_KM_HARDCODED = 695700.0
MOON_RADIUS_KM_HARDCODED = 1737.4
EARTH_EQUATORIAL_RADIUS_KM = 6378.137 # WGS84 value

# Define bodies from the ephemeris
sun = eph['sun']
moon = eph['moon']
earth = eph['earth']

In [ ]:
# Core functions
from turtle import pen


def separation_angle(ts):
    """
    Determine  the angle between sun earth moon . Used in minimisation
    """
    # don't use apparent - this is geometric. eclipselib uses another method altogether
    # numbers withing a few sconds of NASA
    earth_pos = earth.at(ts)
    sun_pos = earth_pos.observe(sun)
    moon_pos = earth_pos.observe(moon)

    separation = sun_pos.separation_from(moon_pos)

    return separation.degrees

def eclipse_possible(ts):
    """
    Determine if the angle between sun earth moon is small enough
    for the chance of eclipse

    Quick and dirty starting points
    """

    return separation_angle(ts) < 2.5

def classify_eclipse(ts):
    """
        Returns umbra, penubra or None if no intersection
    """
    # given positions 
    earth_pos = earth.at(ts)
    sun_pos = earth_pos.observe(sun)
    moon_pos = earth_pos.observe(moon)
    separation = sun_pos.separation_from(moon_pos)

    # given the distance between the sun and moon, and moon and earth, determine the shadow size at earth
    # gives a good estimate, but to aactually classify, you need to know if that shadow hits earth
    # NB: Shadow sizes calculated using similar triangle geoemtry
    D_sun_km = sun_pos.distance().km
    D_moon_km = moon_pos.distance().km
    penumbra_radius_km = D_moon_km * (SUN_RADIUS_KM_HARDCODED + MOON_RADIUS_KM_HARDCODED) / (D_sun_km - D_moon_km)

    # negative umbral radius is annular.
    umbra_radius_km = MOON_RADIUS_KM_HARDCODED - D_moon_km * (SUN_RADIUS_KM_HARDCODED - MOON_RADIUS_KM_HARDCODED) / (D_sun_km - D_moon_km) 

    # Partial eclipse: penumbra centre outside earth
    # Total eclipse: penumbra fully inside earth
    # Annular eclipse: Penumbra inside earth, but negative size

    # calculate distance between centre of penumbra at earth distance, and earth centre
    # TODO: fatten trhshold out and filter for non eclipses here
    eclipse_type = None

    earth_radius_rad = np.arctan2(EARTH_EQUATORIAL_RADIUS_KM, D_moon_km)
    penumbra_radius_rad = np.arctan2(penumbra_radius_km, D_moon_km)
    umbra_radius_rad = np.arctan2(umbra_radius_km, D_moon_km)
    if (separation.radians < penumbra_radius_rad + earth_radius_rad):
        eclipse_type = 'Partial'
    if separation.radians < umbra_radius_rad + earth_radius_rad:
        eclipse_type = "Total"
        if umbra_radius_km < 0:
            eclipse_type ="Annular"

    return eclipse_type

# set default attributes for finding discrete events


def find_plane_and_intersection(ts):
    """
    Returns a scalar representing distance
    """

    earth_pos = earth.at(ts)
    sun_gcrs = earth_pos.observe(sun) # Geometric position is fine here
    moon_gcrs = earth_pos.observe(moon)
    s_vec = sun_gcrs.position.km # Vector Earth->Sun [x,y,z] in GCRS (km)
    m_vec = moon_gcrs.position.km # Vector Earth->Moon [x,y,z] in GCRS (km)

    # Get Shadow axis vector
    L_vec = m_vec - s_vec
    z_fp = L_vec / np.linalg.norm(L_vec, axis=0) # Unit vector along shadow axis (Sun->Moon)
    # This is the Z-axis direction of our fundamental plane coordinate system

    # NCP vector in GCRS is approximately [0, 0, 1]
    NCP_vec = np.array([0.0, 0.0, 1.0])
    NCP_vec = NCP_vec[:, np.newaxis]

    # Project NCP onto the fundamental plane and normalize to get y_fp
    # Handle the case where z_fp is aligned with NCP (axis points poleward)
    # Correct way to calculate dot product for each time point
    dot_prod_z_ncp = np.sum(NCP_vec * z_fp, axis=0) # Result shape: (N,)

    # if abs(abs(dot_prod_z_ncp) - 1.0) < 1e-9: # Axis is aligned with NCP
    #     # Fundamental plane is the equator. Choose X along Vernal Equinox dir.
    #     # Need GCRS X-axis direction vector (approx [1, 0, 0])
    #     GCRS_X_vec = np.array([1.0, 0.0, 0.0])
    #     # Project GCRS X onto the plane (it's already in it if plane is equator)
    #     x_fp = GCRS_X_vec - np.dot(GCRS_X_vec, z_fp) * z_fp # Should be GCRS_X_vec itself
    #     x_fp /= np.linalg.norm(x_fp)
    #     y_fp = np.cross(z_fp, x_fp) # y = z x x gives correct orientation
    # else:
        # Standard case: Project NCP onto plane
        
    y_fp_unnormalized = NCP_vec - (dot_prod_z_ncp * z_fp)
    y_fp = y_fp_unnormalized / np.linalg.norm(y_fp_unnormalized)

    # Define X-axis (`x_fp`) using cross product for a right-handed system
    # Calculate cross product for each time point
    x_fp = np.cross(y_fp, z_fp, axisa=0, axisb=0).T # Transpose to get (3, N)

    return x_fp, y_fp, z_fp






In [74]:
# Test
# Define time range for search

#times, events = almanac.find_discrete(t_start, t_end, eclipse_possible)
from datetime import UTC, timedelta
from skyfield.searchlib import find_maxima, find_minima

# Define time range for search
t_start = ts.utc(1961, 1, 1)
t_end = ts.utc(1970, 12, 31)


separation_angle.rough_period = 1
t, values = find_minima(t_start, t_end, separation_angle)


print(len(t), 'minima found')

mask = eclipse_possible(t)    
t = t[mask]

print(f"Potential Eclipses found: {len(t)}")

eclipses = []
for ti in t:
    kind = classify_eclipse(ti)
    if kind is not None:
        eclipses.append((t,kind))

print(eclipses)

# for ti in t:
#     print(ti.utc_strftime('%Y-%m-%d %H:%M:%S '), classify_eclipse(ti))

124 minima found
Potential Eclipses found: 42


AttributeError: 'Angle' object has no attribute 'all'